<a href="https://colab.research.google.com/github/katayamajunior/Repo_Titanic/blob/Master/Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **RMS Titanic**

O RMS titanic foi um navio de passageiros Britânico, construído em Março de 1909 e lançado ao mar em Maio de 1911. Foi considerado o navio mais sofisticado, luxuoso e seguro de sua época. 
Porém sua fama de "Inafundável", caiu por terra logo em sua viagem inaugural de Southampton com destino a Nova Iorque, em 10 de abril de 1912, 
ao colidir com um Iceberg e naufragar com mais de 1500 pessoas a bordo.
Tornou-se muito famoso, por ser um dos maiores desastres marítimos em tempos de paz de toda história. 
Seu naufrágio destacou vários pontos fracos do projeto, deficiências nos procedimentos de evacuação de emergência e falhas nas regulamentações marítima.
Até hoje seus destroços atrai várias expedições de exploração e estudos.
###### *Fonte:* [WikiPedia](https://pt.wikipedia.org/wiki/RMS_Titanic)

--------


### **O Projeto:**
Com base na história do titanic e de um dataset com informações reais sobre uma parte de seus passageiros,
faremos uma análise exploratória com foco nos sobreviventes, para descobrirmos quais classes, genêros e idades tiveram mais chances, 
assim como a relação dessas três variantes para termos uma idéia comparativa com a nossa própria vida.

A principal pergunta a ser respondida por este projeto: *Naquela época quais as chances de eu ter sobrevivido ao naufrágio?*

### **O DataSet:**

O arquivo titanic.data disponibilizado pela equipe da Udacity, contém informações reais sobre os passageiros organizadas da seguinte forma:

- Survived (Sobreviveu): 0 = Não, 1 = Sim
- Pclass (Classe): Classe de ingresso 1 = 1º, 2 = 2º, 3 = 3º
- Sex (Sexo): Sexo do passageiro
- Age (Idade): Idade em anos
- Sibsp: Quantidade de irmãos / cônjuges a bordo do Titanic
- Parch: Quantidade de pais / crianças a bordo do Titanic
- Ticket (Bilhete): Número do bilhete de embarque
- Fare (Tarifa): Tarifa paga pelo Passageiro
- Cabin (Cabine): Número de cabine
- Embarked (Embarque): Porto de Embarque (C = Cherbourg, Q = Queenstown, S = Southampton)



##### **Importando o DataSet**
1 - Efetuar a leitura do dataset através do Pandas, gerando um DataFrame de nome tit, levando-se em consideração que o arquivo encontra-se na mesma pasta do notebook.

In [3]:
import numpy as np
import pandas as pd 

In [4]:
file = pd.read_csv('titanic.data')

tit = pd.DataFrame(file)

tit.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


##### **Entendendo o DataSet**
1 - Qual a estrutura do dataframe? Quantas colunas e linhas?

In [6]:
tit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


##### **Preparando o DataSet**
1 - Excluindo as colunas que não serão úteis para esta análise. (SibSp, Parch, Ticket, Cabin, Embarked).

In [7]:
tit.drop(columns=['SibSp','Parch','Ticket','Cabin','Embarked'], inplace=True)
tit.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,Fare
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,7.2500
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,71.2833
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,7.9250
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,53.1000
4,5,0,3,"Allen, Mr. William Henry",male,35.0,8.0500
